## 🎯 What's Next?

After exporting your configuration:

1. **Copy the generated code** from the export widget above
2. **Open a simulation notebook** (like `file_import_test.ipynb`)
3. **Paste your configuration code** to replace the manual component setup
4. **Load trajectory data** (GPX files from the `docs/gpx_files/` folder)
5. **Run the simulation** to see how your custom motorcycle performs!

### Example Next Steps:

```python
# In your simulation notebook:

# 1. Load trajectory data
from ithaka_powertrain_sim.trajectory import load_gpx, append_and_resample_dataframe
dataframe = load_gpx("docs/gpx_files/norcal-stanford.gpx")
dataframe = append_and_resample_dataframe(dataframe)

# 2. Paste your exported configuration code here
# (your custom motorcycle setup)

# 3. Run simulation
# (follow the simulation loop from existing notebooks)
```

### Performance Analysis:

The simulation will show you:
- **Energy consumption** over the route
- **Speed performance** vs target speeds  
- **Range estimation** based on energy usage
- **Component utilization** and efficiency

Happy powertrain designing! 🚀

In [ ]:
# Create and display the export widget
export_widget = create_configuration_export(state)
display(export_widget)

## 📤 Export Configuration

Once you're happy with your configuration, export the code to use in other notebooks:

In [ ]:
# Export functionality
export_output = widgets.Output()

def export_results(format_type):
    """Export simulation results in specified format."""
    
    with export_output:
        clear_output(wait=True)
        
        if not simulation_results:
            print("❌ No simulation results to export. Run a simulation first!")
            return
        
        df = simulation_results['dataframe']
        config = simulation_results['config']
        
        try:
            if format_type == 'csv':
                # Export data as CSV
                filename = f"simulation_results_{config['motorcycle_type']}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
                df.to_csv(filename, index=False)
                print(f"✅ CSV data exported to: {filename}")
                
            elif format_type == 'summary':
                # Export summary report as text
                filename = f"simulation_summary_{config['motorcycle_type']}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.txt"
                
                with open(filename, 'w') as f:
                    f.write("ITHAKA POWERTRAIN SIMULATION REPORT\\n")
                    f.write("=" * 50 + "\\n\\n")
                    f.write(f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\\n")
                    f.write(f"Motorcycle Type: {config['motorcycle_type']}\\n")
                    f.write(f"Route: {config['route']}\\n\\n")
                    
                    f.write("CONFIGURATION:\\n")
                    f.write("-" * 20 + "\\n")
                    for comp_type, comp in config['components'].items():
                        if comp:
                            f.write(f"{comp_type.title()}: {comp.name}\\n")
                    f.write("\\n")
                    
                    f.write("RESULTS SUMMARY:\\n")
                    f.write("-" * 20 + "\\n")
                    f.write(f"Total Distance: {df['distance'].iloc[-1]:.1f} km\\n")
                    f.write(f"Total Time: {df['time'].iloc[-1]/60:.1f} minutes\\n")
                    f.write(f"Average Speed: {df['distance'].iloc[-1] / (df['time'].iloc[-1]/3600):.1f} km/h\\n")
                    f.write(f"Energy Consumption: {df['total_energy'].iloc[-1]:.2f} MJ\\n")
                    f.write(f"Energy Efficiency: {df['total_energy'].iloc[-1] / df['distance'].iloc[-1]:.3f} MJ/km\\n")
                    
                print(f"✅ Summary report exported to: {filename}")
                
            elif format_type == 'config':
                # Export configuration as JSON
                filename = f"motorcycle_config_{config['motorcycle_type']}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.json"
                
                # Create exportable config
                export_config = {
                    'motorcycle_type': config['motorcycle_type'],
                    'route': config['route'],
                    'components': {}
                }
                
                for comp_type, comp in config['components'].items():
                    if comp:
                        export_config['components'][comp_type] = {
                            'name': comp.name,
                            'type': comp.__class__.__name__
                        }
                
                import json
                with open(filename, 'w') as f:
                    json.dump(export_config, f, indent=2)
                    
                print(f"✅ Configuration exported to: {filename}")
                
            elif format_type == 'plots':
                # Export plots as high-resolution PNG
                filename = f"simulation_plots_{config['motorcycle_type']}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.png"
                
                # Recreate the plots for export
                fig = plt.figure(figsize=(15, 12))
                
                # Plot 1: Speed Profile
                plt.subplot(2, 3, 1)
                plt.plot(df['distance'], df['target_speed'], 'b--', label='Target Speed', alpha=0.7)
                plt.plot(df['distance'], df['achieved_speed'], 'g-', label='Achieved Speed', linewidth=2)
                plt.xlabel('Distance (km)')
                plt.ylabel('Speed (m/s)')
                plt.title('Speed Performance')
                plt.legend()
                plt.grid(True, alpha=0.3)
                
                # Plot 2: Elevation Profile
                plt.subplot(2, 3, 2)
                plt.plot(df['distance'], df['elevation'], 'brown', linewidth=2)
                plt.xlabel('Distance (km)')
                plt.ylabel('Elevation (m)')
                plt.title('Route Elevation')
                plt.grid(True, alpha=0.3)
                plt.fill_between(df['distance'], df['elevation'], alpha=0.3, color='brown')
                
                # Plot 3: Energy Consumption
                plt.subplot(2, 3, 3)
                plt.plot(df['distance'], df['total_energy'], 'red', linewidth=2)
                plt.xlabel('Distance (km)')
                plt.ylabel('Total Energy (MJ)')
                plt.title('Cumulative Energy Consumption')
                plt.grid(True, alpha=0.3)
                
                # Plot 4: Speed vs Target Speed Scatter
                plt.subplot(2, 3, 4)
                plt.scatter(df['target_speed'], df['achieved_speed'], alpha=0.5, s=1)
                max_speed = max(df['target_speed'].max(), df['achieved_speed'].max())
                plt.plot([0, max_speed], [0, max_speed], 'r--', label='Perfect Match')
                plt.xlabel('Target Speed (m/s)')
                plt.ylabel('Achieved Speed (m/s)')
                plt.title('Speed Achievement')
                plt.legend()
                plt.grid(True, alpha=0.3)
                
                # Plot 5: Energy Rate
                plt.subplot(2, 3, 5)
                energy_rate = df['total_energy'].diff() / df['time'].diff()
                energy_rate = energy_rate.fillna(0)
                plt.plot(df['distance'], energy_rate * 1000, 'orange', linewidth=1)
                plt.xlabel('Distance (km)')
                plt.ylabel('Power (kW)')
                plt.title('Instantaneous Power')
                plt.grid(True, alpha=0.3)
                
                # Plot 6: Summary Info
                plt.subplot(2, 3, 6)
                plt.text(0.1, 0.9, f"Motorcycle: {config['motorcycle_type']}", transform=plt.gca().transAxes, fontsize=12, weight='bold')
                plt.text(0.1, 0.8, f"Route: {config['route']}", transform=plt.gca().transAxes, fontsize=10)
                plt.text(0.1, 0.7, f"Distance: {df['distance'].iloc[-1]:.1f} km", transform=plt.gca().transAxes, fontsize=10)
                plt.text(0.1, 0.6, f"Time: {df['time'].iloc[-1]/60:.1f} min", transform=plt.gca().transAxes, fontsize=10)
                plt.text(0.1, 0.5, f"Energy: {df['total_energy'].iloc[-1]:.2f} MJ", transform=plt.gca().transAxes, fontsize=10)
                plt.text(0.1, 0.4, f"Efficiency: {df['total_energy'].iloc[-1] / df['distance'].iloc[-1]:.3f} MJ/km", transform=plt.gca().transAxes, fontsize=10)
                plt.text(0.1, 0.3, f"Avg Speed: {df['distance'].iloc[-1] / (df['time'].iloc[-1]/3600):.1f} km/h", transform=plt.gca().transAxes, fontsize=10)
                plt.title('Simulation Summary')
                plt.axis('off')
                
                plt.suptitle(f'Ithaka Powertrain Simulation - {config["motorcycle_type"]}', fontsize=16, weight='bold')
                plt.tight_layout()
                plt.savefig(filename, dpi=300, bbox_inches='tight')
                plt.close()
                
                print(f"✅ High-resolution plots exported to: {filename}")
                
        except Exception as e:
            print(f"❌ Export failed: {str(e)}")

# Create export buttons
export_buttons = [
    widgets.Button(description='📊 Export CSV Data', button_style='', layout=widgets.Layout(width='160px')),
    widgets.Button(description='📄 Export Summary', button_style='', layout=widgets.Layout(width='160px')),
    widgets.Button(description='⚙️ Export Config', button_style='', layout=widgets.Layout(width='160px')),
    widgets.Button(description='📈 Export Plots', button_style='', layout=widgets.Layout(width='160px'))
]

# Set up button callbacks
export_buttons[0].on_click(lambda b: export_results('csv'))
export_buttons[1].on_click(lambda b: export_results('summary'))
export_buttons[2].on_click(lambda b: export_results('config'))
export_buttons[3].on_click(lambda b: export_results('plots'))

# Display export interface
export_widget = widgets.VBox([
    widgets.HTML("<p><b>Choose export format:</b></p>"),
    widgets.HBox(export_buttons[:2]),
    widgets.HBox(export_buttons[2:]),
    export_output
])

display(export_widget)

## 📄 Export Results

Save your simulation results in various formats for reports, presentations, or further analysis:

In [ ]:
# Results visualization and analysis
results_output = widgets.Output()

def display_results():
    """Display comprehensive simulation results with visualizations."""
    
    with results_output:
        clear_output(wait=True)
        
        if not simulation_results:
            print("🤔 No simulation results yet. Run a simulation first!")
            return
        
        df = simulation_results['dataframe']
        config = simulation_results['config']
        motorbike = simulation_results['motorbike']
        
        print("🎯 SIMULATION SUMMARY")
        print("=" * 50)
        print(f"Motorcycle Type: {config['motorcycle_type']}")
        print(f"Route: {config['route']}")
        print(f"Total Distance: {df['distance'].iloc[-1]:.1f} km")
        print(f"Total Time: {df['time'].iloc[-1]/60:.1f} minutes")
        print(f"Average Speed: {df['distance'].iloc[-1] / (df['time'].iloc[-1]/3600):.1f} km/h")
        print(f"Final Energy Consumption: {df['total_energy'].iloc[-1]:.2f} MJ")
        print()
        
        # Create comprehensive plots
        fig = plt.figure(figsize=(15, 12))
        
        # Plot 1: Speed Profile
        plt.subplot(2, 3, 1)
        plt.plot(df['distance'], df['target_speed'], 'b--', label='Target Speed', alpha=0.7)
        plt.plot(df['distance'], df['achieved_speed'], 'g-', label='Achieved Speed', linewidth=2)
        plt.xlabel('Distance (km)')
        plt.ylabel('Speed (m/s)')
        plt.title('Speed Performance')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Plot 2: Elevation Profile
        plt.subplot(2, 3, 2)
        plt.plot(df['distance'], df['elevation'], 'brown', linewidth=2)
        plt.xlabel('Distance (km)')
        plt.ylabel('Elevation (m)')
        plt.title('Route Elevation')
        plt.grid(True, alpha=0.3)
        plt.fill_between(df['distance'], df['elevation'], alpha=0.3, color='brown')
        
        # Plot 3: Energy Consumption
        plt.subplot(2, 3, 3)
        plt.plot(df['distance'], df['total_energy'], 'red', linewidth=2)
        plt.xlabel('Distance (km)')
        plt.ylabel('Total Energy (MJ)')
        plt.title('Cumulative Energy Consumption')
        plt.grid(True, alpha=0.3)
        
        # Plot 4: Speed vs Target Speed Scatter
        plt.subplot(2, 3, 4)
        plt.scatter(df['target_speed'], df['achieved_speed'], alpha=0.5, s=1)
        max_speed = max(df['target_speed'].max(), df['achieved_speed'].max())
        plt.plot([0, max_speed], [0, max_speed], 'r--', label='Perfect Match')
        plt.xlabel('Target Speed (m/s)')
        plt.ylabel('Achieved Speed (m/s)')
        plt.title('Speed Achievement')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Plot 5: Energy Rate
        plt.subplot(2, 3, 5)
        energy_rate = df['total_energy'].diff() / df['time'].diff()  # MJ/s = MW
        energy_rate = energy_rate.fillna(0)
        plt.plot(df['distance'], energy_rate * 1000, 'orange', linewidth=1)  # Convert to kW
        plt.xlabel('Distance (km)')
        plt.ylabel('Power (kW)')
        plt.title('Instantaneous Power')
        plt.grid(True, alpha=0.3)
        
        # Plot 6: Performance Metrics
        plt.subplot(2, 3, 6)
        # Calculate some key metrics
        speed_error = abs(df['achieved_speed'] - df['target_speed'])
        avg_speed_error = speed_error.mean()
        max_speed_error = speed_error.max()
        
        metrics = ['Avg Speed\nError (m/s)', 'Max Speed\nError (m/s)', 'Final Energy\n(MJ)', 'Avg Power\n(kW)']
        values = [
            avg_speed_error,
            max_speed_error, 
            df['total_energy'].iloc[-1],
            (df['total_energy'].iloc[-1] / (df['time'].iloc[-1]/3600)) * 1000/3600  # Convert to kW
        ]
        
        colors = ['lightblue', 'lightcoral', 'lightgreen', 'lightyellow']
        bars = plt.bar(metrics, values, color=colors)
        plt.title('Key Performance Metrics')
        plt.xticks(rotation=45)
        
        # Add value labels on bars
        for bar, value in zip(bars, values):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01*max(values),
                    f'{value:.2f}', ha='center', va='bottom', fontsize=9)
        
        plt.tight_layout()
        plt.show()
        
        # Performance Analysis
        print()
        print("📈 PERFORMANCE ANALYSIS")
        print("=" * 50)
        
        # Speed performance
        speed_achievement = (1 - speed_error.mean() / df['target_speed'].mean()) * 100
        print(f"Speed Achievement: {speed_achievement:.1f}%")
        
        if speed_achievement > 95:
            print("   ✅ Excellent speed performance!")
        elif speed_achievement > 85:
            print("   👍 Good speed performance")
        else:
            print("   ⚠️  Speed performance could be improved")
        
        # Energy efficiency
        distance_total = df['distance'].iloc[-1]
        energy_per_km = df['total_energy'].iloc[-1] / distance_total
        print(f"Energy Efficiency: {energy_per_km:.3f} MJ/km ({energy_per_km*0.278:.2f} kWh/km)")
        
        # Component utilization
        print()
        print("🔧 COMPONENT ANALYSIS")
        print("=" * 30)
        
        # Analyze energy sources
        for source in motorbike.energy_sources:
            if hasattr(source, 'current_fuel_level_L'):
                fuel_used = source.capacity_L - source.current_fuel_level_L
                fuel_efficiency = distance_total / fuel_used if fuel_used > 0 else float('inf')
                print(f"Fuel Tank: {fuel_used:.2f}L used ({fuel_efficiency:.1f} km/L)")
                
            elif hasattr(source, 'current_charge_Ah'):
                charge_used = source.capacity_Ah - source.current_charge_Ah
                charge_percent = (charge_used / source.capacity_Ah) * 100
                print(f"Battery: {charge_used:.1f}Ah used ({charge_percent:.1f}% of capacity)")
        
        print()
        print("💡 RECOMMENDATIONS")
        print("=" * 30)
        
        # Generate recommendations based on results
        recommendations = []
        
        if speed_achievement < 90:
            recommendations.append("Consider more powerful motor/engine for better speed performance")
        
        if energy_per_km > 2.0:  # High energy consumption
            recommendations.append("Energy consumption is high - consider lighter components or better aerodynamics")
        
        if max_speed_error > 5:
            recommendations.append("Large speed variations detected - check power-to-weight ratio")
            
        # Check for battery/fuel depletion
        for source in motorbike.energy_sources:
            if hasattr(source, 'current_charge_Ah'):
                if source.current_charge_Ah / source.capacity_Ah < 0.1:
                    recommendations.append("Battery nearly depleted - consider larger battery capacity")
            elif hasattr(source, 'current_fuel_level_L'):
                if source.current_fuel_level_L / source.capacity_L < 0.1:
                    recommendations.append("Fuel nearly depleted - consider larger fuel tank")
        
        if not recommendations:
            recommendations.append("Great configuration! Performance looks optimal.")
        
        for i, rec in enumerate(recommendations, 1):
            print(f"{i}. {rec}")

# Auto-display results when available
def check_and_display_results():
    """Check if results are available and display them."""
    if simulation_results:
        display_results()

# Create results display
display(results_output)

# Note: Results will be displayed automatically after simulation completes

In [ ]:
# Add a button to manually refresh results display
refresh_button = widgets.Button(
    description='🔄 Show Results',
    button_style='info',
    layout=widgets.Layout(width='150px'),
)

refresh_button.on_click(lambda b: display_results())

display(refresh_button)

## 📊 Simulation Results & Analysis

After running a simulation, detailed results and visualizations will appear here:

# Ithaka Motorcycle Component tester

- **Select from preset components** with proven specifications
- **Create custom components** with your own parameters
- **See real-time feedback** on weight, cost, and performance
- **Export configurations** ready for simulation

Let's build your motorcycle powertrain!

In [ ]:
#@title 🚀 Environment Setup for Google Colab { display-mode: "form" }

#@markdown **For Google Colab users only** - This cell automatically sets up the environment.
#@markdown **Local users can skip this cell** - just run the cell below instead.

import sys
import os

def setup_colab_environment():
    """Smart setup for Google Colab that detects existing repositories."""
    
    try:
        import google.colab
        print("📦 Google Colab environment detected")
    except ImportError:
        print("💻 Not in Colab - skip this cell if running locally")
        return False
    
    print("🔍 Checking for existing repository...")
    
    # Check if we're already in a valid repository
    current_dir = os.getcwd()
    possible_repo_paths = [
        '/content/Teo-Moto-Repo',
        '/content/ithaka-powertrain-sim', 
        current_dir
    ]
    
    repo_found = False
    for path in possible_repo_paths:
        if os.path.exists(path):
            # Check if it contains our package
            setup_py_path = os.path.join(path, 'setup.py')
            requirements_path = os.path.join(path, 'requirements.txt')
            if os.path.exists(setup_py_path) or os.path.exists(requirements_path):
                print(f"✅ Found existing repository at: {path}")
                os.chdir(path)
                repo_found = True
                break
    
    if not repo_found:
        print("📥 Cloning repository from GitHub...")
        
        # Try the correct repository URL
        repo_urls = [
            'https://github.com/Teodus/Teo-Moto-Repo.git',
            'https://github.com/Teodus/ithaka-powertrain-sim.git'
        ]
        
        clone_success = False
        for repo_url in repo_urls:
            try:
                print(f"   Trying: {repo_url}")
                result = os.system(f'git clone {repo_url}')
                if result == 0:  # Success
                    # Extract repo name from URL
                    repo_name = repo_url.split('/')[-1].replace('.git', '')
                    repo_path = f'/content/{repo_name}'
                    
                    if os.path.exists(repo_path):
                        print(f"✅ Successfully cloned to: {repo_path}")
                        os.chdir(repo_path)
                        clone_success = True
                        break
            except Exception as e:
                print(f"   ❌ Failed: {e}")
                continue
        
        if not clone_success:
            print("❌ Could not clone repository. Please check your internet connection.")
            return False
    
    # Install requirements
    print("📦 Installing requirements...")
    if os.path.exists('requirements.txt'):
        os.system('pip install -r requirements.txt -q')
        print("✅ Requirements installed")
    else:
        print("⚠️ No requirements.txt found, skipping package installation")
    
    print("🎉 Setup complete!")
    return True

# Run setup
setup_colab_environment()

In [ ]:
#@title 📦 Import Required Libraries { display-mode: "form" }

#@markdown This cell imports all necessary libraries and components.
#@markdown **Run this cell** after the setup above completes successfully.

import ipywidgets as widgets
from IPython.display import display, clear_output

# Import our component picker utilities
from ithaka_powertrain_sim.component_library.interactive_picker import (
    ComponentPickerState,
    create_motorcycle_type_selector,
    create_engine_selector,
    create_motor_selector,
    create_battery_selector,
    create_metrics_display,
    create_configuration_export
)

print("✅ Interactive component picker loaded successfully!")

## 🔧 Component Selection Interface

Use the widgets below to select or create your motorcycle components. The interface will provide real-time feedback on your choices and validate component compatibility.

In [ ]:
#@title 🎛️ Initialize Component Selectors { display-mode: "form" }

#@markdown This cell creates all the interactive component selection widgets.
#@markdown **Run this cell** to initialize the component picker interface.

# Initialize the component picker state
state = ComponentPickerState()

# Create metrics display
metrics_widget = create_metrics_display(state)

# Define update callback for real-time metrics
def update_metrics():
    state.update_metrics()
    metrics_widget.update_display()

# Create component selectors
type_selector = create_motorcycle_type_selector(state, update_metrics)
engine_selector = create_engine_selector(state, update_metrics)
motor_selector = create_motor_selector(state, update_metrics)
battery_selector = create_battery_selector(state, update_metrics)

print("🎛️ Component selectors initialized successfully!")

### Step 1: Choose Your Motorcycle Type

In [11]:
display(type_selector)

RadioButtons(description='Motorcycle Type:', options=('Pure EV', 'Hybrid', 'Pure ICE'), style=DescriptionStyle…

### Step 2: Select Engine (for ICE and Hybrid motorcycles)

Choose from preset engines or create a custom engine with your specifications:

In [12]:
display(engine_selector)

### Step 3: Select Electric Motor (for EV and Hybrid motorcycles)

Choose from preset motors or create a custom motor with your specifications:

In [13]:
display(motor_selector)

### Step 4: Select Battery Pack (for EV and Hybrid motorcycles)

Choose from preset battery packs or create a custom pack with your specifications:

In [14]:
display(battery_selector)

## 📊 Real-time Configuration Metrics

This panel shows live updates of your motorcycle's specifications as you make component selections:

In [15]:
display(metrics_widget)

HTML(value='\n        <div style="background-color: #f0f0f0; padding: 15px; border-radius: 5px; margin: 10px 0…

## ⚠️ Configuration Validation

Let's check if your current configuration makes sense:

In [ ]:
#@title 🔍 Configuration Validation { display-mode: "form" }

#@markdown This cell validates your current motorcycle configuration and provides recommendations.
#@markdown **Run this cell** after making your component selections to check for issues.

def validate_configuration(state):
    """Validate the current motorcycle configuration."""
    warnings = []
    errors = []

    # Check motorcycle type requirements
    if state.motorcycle_type == "Pure EV":
        if not state.selected_components.get('motor'):
            errors.append("Pure EV requires an electric motor")
        if not state.selected_components.get('battery'):
            errors.append("Pure EV requires a battery pack")
        if state.selected_components.get('engine'):
            warnings.append("Pure EV shouldn't have an engine - consider Hybrid instead")

    elif state.motorcycle_type == "Pure ICE":
        if not state.selected_components.get('engine'):
            errors.append("Pure ICE requires an engine")
        if state.selected_components.get('motor'):
            warnings.append("Pure ICE shouldn't have an electric motor - consider Hybrid instead")
        if state.selected_components.get('battery'):
            warnings.append("Pure ICE shouldn't have a battery pack - consider Hybrid instead")

    elif state.motorcycle_type == "Hybrid":
        if not state.selected_components.get('engine'):
            errors.append("Hybrid requires an engine")
        if not state.selected_components.get('motor'):
            errors.append("Hybrid requires an electric motor")
        if not state.selected_components.get('battery'):
            errors.append("Hybrid requires a battery pack")

    # Check power balance
    motor = state.selected_components.get('motor')
    engine = state.selected_components.get('engine')

    if motor and engine:
        motor_power = motor._maximum_power_generation / 1000  # Convert to kW
        engine_power = engine._maximum_power_generation / 1000

        if motor_power > engine_power * 3:
            warnings.append(f"Motor ({motor_power:.0f}kW) much more powerful than engine ({engine_power:.0f}kW)")
        elif engine_power > motor_power * 3:
            warnings.append(f"Engine ({engine_power:.0f}kW) much more powerful than motor ({motor_power:.0f}kW)")

    # Check weight distribution
    if state.total_weight > 350:
        warnings.append(f"Total weight ({state.total_weight:.0f}kg) is quite heavy for a motorcycle")
    elif state.total_weight < 120:
        warnings.append(f"Total weight ({state.total_weight:.0f}kg) is quite light - double-check component masses")

    # Check power-to-weight ratio
    if state.power_to_weight > 0.4:
        warnings.append(f"Power-to-weight ratio ({state.power_to_weight:.3f} kW/kg) is very high - superbike performance!")
    elif state.power_to_weight < 0.1:
        warnings.append(f"Power-to-weight ratio ({state.power_to_weight:.3f} kW/kg) is quite low")

    return errors, warnings

# Run validation
errors, warnings = validate_configuration(state)

print("🔍 Configuration Validation Results:")
print("=" * 40)

if not errors and not warnings:
    print("✅ Configuration looks good!")
else:
    if errors:
        print("❌ Errors (must be fixed):")
        for error in errors:
            print(f"   • {error}")
        print()

    if warnings:
        print("⚠️ Warnings (consider reviewing):")
        for warning in warnings:
            print(f"   • {warning}")

if not errors:
    print("\\n🚀 Ready to export configuration!")

In [ ]:
#@title 📤 Export Configuration { display-mode: "form" }

#@markdown This cell creates the export widget to generate code for your motorcycle configuration.
#@markdown **Run this cell** when you're ready to export your design.

# Create and display the export widget
export_widget = create_configuration_export(state)
display(export_widget)

In [ ]:
# Export functionality
export_output = widgets.Output()

def export_results(format_type):
    """Export simulation results in specified format."""
    
    with export_output:
        clear_output(wait=True)
        
        if not simulation_results:
            print("❌ No simulation results to export. Run a simulation first!")
            return
        
        df = simulation_results['dataframe']
        config = simulation_results['config']
        
        try:
            if format_type == 'csv':
                # Export data as CSV
                filename = f"simulation_results_{config['motorcycle_type']}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
                df.to_csv(filename, index=False)
                print(f"✅ CSV data exported to: {filename}")
                
            elif format_type == 'summary':
                # Export summary report as text
                filename = f"simulation_summary_{config['motorcycle_type']}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.txt"
                
                with open(filename, 'w') as f:
                    f.write("ITHAKA POWERTRAIN SIMULATION REPORT\\n")
                    f.write("=" * 50 + "\\n\\n")
                    f.write(f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\\n")
                    f.write(f"Motorcycle Type: {config['motorcycle_type']}\\n")
                    f.write(f"Route: {config['route']}\\n\\n")
                    
                    f.write("CONFIGURATION:\\n")
                    f.write("-" * 20 + "\\n")
                    for comp_type, comp in config['components'].items():
                        if comp:
                            f.write(f"{comp_type.title()}: {comp.name}\\n")
                    f.write("\\n")
                    
                    f.write("RESULTS SUMMARY:\\n")
                    f.write("-" * 20 + "\\n")
                    f.write(f"Total Distance: {df['distance'].iloc[-1]:.1f} km\\n")
                    f.write(f"Total Time: {df['time'].iloc[-1]/60:.1f} minutes\\n")
                    f.write(f"Average Speed: {df['distance'].iloc[-1] / (df['time'].iloc[-1]/3600):.1f} km/h\\n")
                    f.write(f"Energy Consumption: {df['total_energy'].iloc[-1]:.2f} MJ\\n")
                    f.write(f"Energy Efficiency: {df['total_energy'].iloc[-1] / df['distance'].iloc[-1]:.3f} MJ/km\\n")
                    
                print(f"✅ Summary report exported to: {filename}")
                
            elif format_type == 'config':
                # Export configuration as JSON
                filename = f"motorcycle_config_{config['motorcycle_type']}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.json"
                
                # Create exportable config
                export_config = {
                    'motorcycle_type': config['motorcycle_type'],
                    'route': config['route'],
                    'components': {}
                }
                
                for comp_type, comp in config['components'].items():
                    if comp:
                        export_config['components'][comp_type] = {
                            'name': comp.name,
                            'type': comp.__class__.__name__
                        }
                
                import json
                with open(filename, 'w') as f:
                    json.dump(export_config, f, indent=2)
                    
                print(f"✅ Configuration exported to: {filename}")
                
            elif format_type == 'plots':
                # Export plots as high-resolution PNG
                filename = f"simulation_plots_{config['motorcycle_type']}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.png"
                
                # Recreate the plots for export
                fig = plt.figure(figsize=(15, 12))
                
                # Plot 1: Speed Profile
                plt.subplot(2, 3, 1)
                plt.plot(df['distance'], df['target_speed'], 'b--', label='Target Speed', alpha=0.7)
                plt.plot(df['distance'], df['achieved_speed'], 'g-', label='Achieved Speed', linewidth=2)
                plt.xlabel('Distance (km)')
                plt.ylabel('Speed (m/s)')
                plt.title('Speed Performance')
                plt.legend()
                plt.grid(True, alpha=0.3)
                
                # Plot 2: Elevation Profile
                plt.subplot(2, 3, 2)
                plt.plot(df['distance'], df['elevation'], 'brown', linewidth=2)
                plt.xlabel('Distance (km)')
                plt.ylabel('Elevation (m)')
                plt.title('Route Elevation')
                plt.grid(True, alpha=0.3)
                plt.fill_between(df['distance'], df['elevation'], alpha=0.3, color='brown')
                
                # Plot 3: Energy Consumption
                plt.subplot(2, 3, 3)
                plt.plot(df['distance'], df['total_energy'], 'red', linewidth=2)
                plt.xlabel('Distance (km)')
                plt.ylabel('Total Energy (MJ)')
                plt.title('Cumulative Energy Consumption')
                plt.grid(True, alpha=0.3)
                
                # Plot 4: Speed vs Target Speed Scatter
                plt.subplot(2, 3, 4)
                plt.scatter(df['target_speed'], df['achieved_speed'], alpha=0.5, s=1)
                max_speed = max(df['target_speed'].max(), df['achieved_speed'].max())
                plt.plot([0, max_speed], [0, max_speed], 'r--', label='Perfect Match')
                plt.xlabel('Target Speed (m/s)')
                plt.ylabel('Achieved Speed (m/s)')
                plt.title('Speed Achievement')
                plt.legend()
                plt.grid(True, alpha=0.3)
                
                # Plot 5: Energy Rate
                plt.subplot(2, 3, 5)
                energy_rate = df['total_energy'].diff() / df['time'].diff()
                energy_rate = energy_rate.fillna(0)
                plt.plot(df['distance'], energy_rate * 1000, 'orange', linewidth=1)
                plt.xlabel('Distance (km)')
                plt.ylabel('Power (kW)')
                plt.title('Instantaneous Power')
                plt.grid(True, alpha=0.3)
                
                # Plot 6: Summary Info
                plt.subplot(2, 3, 6)
                plt.text(0.1, 0.9, f"Motorcycle: {config['motorcycle_type']}", transform=plt.gca().transAxes, fontsize=12, weight='bold')
                plt.text(0.1, 0.8, f"Route: {config['route']}", transform=plt.gca().transAxes, fontsize=10)
                plt.text(0.1, 0.7, f"Distance: {df['distance'].iloc[-1]:.1f} km", transform=plt.gca().transAxes, fontsize=10)
                plt.text(0.1, 0.6, f"Time: {df['time'].iloc[-1]/60:.1f} min", transform=plt.gca().transAxes, fontsize=10)
                plt.text(0.1, 0.5, f"Energy: {df['total_energy'].iloc[-1]:.2f} MJ", transform=plt.gca().transAxes, fontsize=10)
                plt.text(0.1, 0.4, f"Efficiency: {df['total_energy'].iloc[-1] / df['distance'].iloc[-1]:.3f} MJ/km", transform=plt.gca().transAxes, fontsize=10)
                plt.text(0.1, 0.3, f"Avg Speed: {df['distance'].iloc[-1] / (df['time'].iloc[-1]/3600):.1f} km/h", transform=plt.gca().transAxes, fontsize=10)
                plt.title('Simulation Summary')
                plt.axis('off')
                
                plt.suptitle(f'Ithaka Powertrain Simulation - {config["motorcycle_type"]}', fontsize=16, weight='bold')
                plt.tight_layout()
                plt.savefig(filename, dpi=300, bbox_inches='tight')
                plt.close()
                
                print(f"✅ High-resolution plots exported to: {filename}")
                
        except Exception as e:
            print(f"❌ Export failed: {str(e)}")

# Create export buttons
export_buttons = [
    widgets.Button(description='📊 Export CSV Data', button_style='', layout=widgets.Layout(width='160px')),
    widgets.Button(description='📄 Export Summary', button_style='', layout=widgets.Layout(width='160px')),
    widgets.Button(description='⚙️ Export Config', button_style='', layout=widgets.Layout(width='160px')),
    widgets.Button(description='📈 Export Plots', button_style='', layout=widgets.Layout(width='160px'))
]

# Set up button callbacks
export_buttons[0].on_click(lambda b: export_results('csv'))
export_buttons[1].on_click(lambda b: export_results('summary'))
export_buttons[2].on_click(lambda b: export_results('config'))
export_buttons[3].on_click(lambda b: export_results('plots'))

# Display export interface
export_widget = widgets.VBox([
    widgets.HTML("<p><b>Choose export format:</b></p>"),
    widgets.HBox(export_buttons[:2]),
    widgets.HBox(export_buttons[2:]),
    export_output
])

display(export_widget)

In [ ]:
# Add a button to manually refresh results display
refresh_button = widgets.Button(
    description='🔄 Show Results',
    button_style='info',
    layout=widgets.Layout(width='150px'),
)

refresh_button.on_click(lambda b: display_results())

display(refresh_button)

## 📊 Simulation Results & Analysis

After running a simulation, detailed results and visualizations will appear here:

### 🎬 Run Your Simulation

Ready to see how your motorcycle performs? Click the button below to run the complete simulation!

### Step 5: Choose Your Test Route

Select from real-world GPS tracks to test your motorcycle's performance:

## 🏁 Integrated Simulation & Analysis

Now let's test your motorcycle configuration on real-world routes! No need to copy code or switch notebooks - everything happens right here.

## 💡 Tips for Component Selection

### Engine Selection:
- **Small engines (250-400cc)**: Good for city commuting, lower weight
- **Medium engines (500-650cc)**: Balanced performance for mixed riding
- **Large engines (750cc+)**: High performance, highway capability

### Motor Selection:
- **Hub motors**: Simple, integrated into wheel, lower power
- **Mid-drive motors**: Better torque characteristics, more complex
- **High-performance motors**: Maximum power, sport riding capability

### Battery Selection:
- **Energy density**: Higher Wh/kg = lighter for same capacity
- **Power density**: Higher C-rate = better acceleration/regen
- **Capacity**: More kWh = longer range but heavier

### Configuration Types:
- **Pure EV**: Zero emissions, quiet, instant torque, limited range
- **Pure ICE**: Long range, quick refueling, familiar technology
- **Hybrid**: Best of both worlds, complex control systems

### Trade-offs to Consider:
- **Weight vs Range**: More battery = more range but heavier bike
- **Power vs Efficiency**: Higher power often means lower efficiency
- **Cost vs Performance**: Premium components cost more but perform better
- **Complexity vs Reliability**: Simpler systems are often more reliable